In [1]:
# Import packages
import poreana as pa
import porems as pms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
import MDAnalysis as mda
import nglview as nv
from IPython.display import display, Markdown, Latex


In [2]:
# Set strings to the files in data set
def set_strings(molecule):

    strings = {}
    for mol in molecule:
        strings[mol] = {}
        # Strcuture for molecule
        if molecule == "water":
            strings[mol]["gro_mol"] = "Molecules/water/tip4p2005.gro"
        elif molecule != "water":
            strings[mol]["gro_mol"] = "Molecules/{}/{}.gro".format(mol,mol)

        # Strings for pure rectangular systems
        strings[mol]["dens_box"] = "Pure/{}/box_rectangular_{}_density.h5".format(mol,mol)
        strings[mol]["mc_box"] = "Pure/{}/box_rectangular_{}_diffusion_smoluchowski.h5".format(mol,mol)
        
        # Strings for pore system
        strings[mol]["dens"] = "Pore/{}/pore_ideal_{}_density.h5".format(mol,mol)
        strings[mol]["bin"] = "Pore/{}/pore_ideal_{}_diffusion_einstein.h5".format(mol,mol)
        strings[mol]["mc"] = "Pore/{}/pore_ideal_{}_diffusion_smoluchowski.h5".format(mol,mol)

        # Strings for amoprh pore systems (if exisit in dataset)
        if mol in ["cyclopentane","hexane","heptane"]:
            strings[mol]["dens_a"] = "Pore/{}/pore_amorph_{}_density.h5".format(mol,mol)
            strings[mol]["bin_a"] = "Pore/{}/pore_amorph_{}_diffusion_einstein.h5".format(mol,mol)
            strings[mol]["mc_a"] = "Pore/{}/pore_amorph_{}_diffusion_smoluchowski.h5".format(mol,mol)
    return strings

In [3]:
# Display molcule and system
def display_molecule(strings):
    for mol in strings:
        # Set markdown header
        display(Markdown('# Molecule'))

        # Load and display gro file of the molecule and
        mol = mda.Universe(strings[mol]["gro_mol"])
        view = nv.show_mdanalysis(mol)
        display(view)


In [4]:
# Plot density function
def plot_density(strings, amorph):
    color= sns.color_palette("deep")
    # Markdown header
    display(Markdown('# Density'))
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    legend = []
    for mol,i in zip(strings, range(len(strings))):
        # Plot settings
        plt.subplot(1,2,1)
        plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
        plt.xlabel("Reservoir length (nm)")

        plt.xlim([0,10])
        plt.title("Reservoir density")

        # Plot reservoir density
        dens = pa.density.bins(strings[mol]["dens"], is_print=False)
        if len(strings) != 1:
            plt.plot([0],[-1], label = mol, color = color[i] )

        plt.plot(dens["sample"]["data"]["ex_width"],[den for den in dens["num_dens"]["ex"]], label = None, color = color[i] )
        # Check if results for amorph pore exists 
        if amorph and ("dens_a" in strings[mol]):
            dens = pa.density.bins(strings[mol]["dens_a"], is_print=False)
            plt.plot(dens["sample"]["data"]["ex_width"],[den for den in dens["num_dens"]["ex"]],linestyle="--" ,label = "Reservoir Density (Amorph)", color = color[i])
        ax = sns.lineplot(x = dens["sample"]["data"]["ex_width"],y =[dens["mean"]["ex"] for i in dens["num_dens"]["ex"]], label = None,linestyle = "--",  color = color[i])
    if len(strings) != 1:
        plt.legend()
    else:
        if amorph:
            legend = ["Reservoir Density","Mean Reservoir Density"]
        else: 
            legend = ["Reservoir Density","Reservoir Density (Amorph)","Mean Reservoir Density"]
        plt.legend(legend)
    ax.set_ylim(bottom=0)



    for mol,i in zip(strings, range(len(strings))):    
        plt.subplot(1,2,2)

        plt.title("Radial Pore Density")
        plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
        plt.xlabel("Distance from pore center (nm)")
        plt.xlim([0,2.5])
        if len(strings) != 1:
            plt.plot([0],[-1], label = mol, color = color[i] )
        # Plot radial pore density
        dens = pa.density.bins(strings[mol]["dens"], is_print=False)
        ax = sns.lineplot(x = dens["sample"]["data"]["in_width"][:-1],y = [den for den in dens["num_dens"]["in"]], label = None, color = color[i])
        if amorph and ("dens_a" in strings[mol]):
            dens = pa.density.bins(strings[mol]["dens_a"], is_print=False)
            plt.plot(dens["sample"]["data"]["in_width"][:-1],[den for den in dens["num_dens"]["in"]],linestyle="dotted" ,label = "Reservoir Density (Amorph)", color = color[i])
        sns.lineplot(x=dens["sample"]["data"]["ex_width"][:-1],y=[dens["mean"]["ex"] for i in dens["num_dens"]["ex"][:-1]], linestyle = "--", label = None, color = color[i])
        ax.set_ylim(bottom=0)

    if len(strings) != 1:
        plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label= "Silanol")
        plt.legend()
    else:
        plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label= None)
        if amorph:
            legend = ["Pore Density", "Silanol O","Pore Density (Amorph)","Mean Reservoir Density"]
        else:
            legend = ["Pore Density", "Silanol O","Mean Reservoir Density"]
        plt.legend(legend)
    plt.tight_layout()
    plt.show()

In [5]:
# Diffusion (BIN + MC)
def plot_mc_diffusion(strings, amorph):
    kwargs = {}
    legend = []
    kwargs["molecule"] = []
    kwargs["groups"] = []
    plt.figure(figsize=(14,4))
    color= sns.color_palette("deep")
    display(Markdown('# Diffusion Profile'))
    legend = []
    for mol,i in zip(strings, range(len(strings))):
        # Plot bin diffusion
        plt.subplot(1,2,1)
        plt.title("Bin diffusion")
        mc_box = pa.diffusion.mc_fit(strings[mol]["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
        diff_bin = pa.diffusion.bins(strings[mol]["bin"])
        ax  =sns.lineplot(x=np.linspace(0,2.51,100),y=[mc_box[0] for i in range(100)], linestyle="--",label = None, color= color[i])
        pa.diffusion.bins_plot(diff_bin, kwargs = {"color": color[i],"label": None})
        if amorph and ("dens_a" in strings[mol]):
            diff_bin = pa.diffusion.bins(strings[mol]["bin_a"])
            pa.diffusion.bins_plot(diff_bin, kwargs = {"linestyle": "dotted","label": "Bin diffusion (Amorph)", "color": color[i]})
        
        plt.xlim([0,2.5])
        ax.set_ylim(bottom=-0.03)
        dens = pa.density.bins(strings[mol]["dens"], is_print=False)
        diff_bin_mean = pa.diffusion.mean(diff_bin, dens, is_print=False)
        plt.plot([-1],[0], label = mol, color = color[i] )
        
       
    if len(strings) != 1:
        plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label = "Silanol")   
        plt.legend(loc=1)
    elif len(strings) == 1:
        plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label = None) 
        plt.text(0.05, 0.1, 'NVT  : D =' + "%.2f" % (mc_box[1]) +"$\\cdot 10^{-9}$ \nPore : D =" + "%.2f" % (diff_bin_mean) +"$\\cdot 10^{-9}$"  ,  bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 4})
        if amorph:
            legend = ["Box diffusion", "Silanol O",  "Bin diffusion"]
        else:
            legend = ["Box diffusion", "Silanol O",  "Bin diffusion", "Bin diffusion (Amorph)"]
        plt.legend(legend,loc=1) 

    for mol,i in zip(strings, range(len(strings))):
        plt.subplot(1,2,2)
        
        plt.title("MC Diffusion")
        #plt.suptitle("Diffusion calculated witht the Smoluchowski equation ")
        mc_box = pa.diffusion.mc_fit(strings[mol]["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
        if len(strings) == 1:
            mc_pore = pa.diffusion.mc_profile(strings[mol]["mc"], len_step = [10,20,30,40,50,60], kwargs={"label": None, "color": color[i]})
        elif len(strings) != 1:
            mc_pore = pa.diffusion.mc_profile(strings[mol]["mc"], len_step = [10,20,30,40,50,60], kwargs={"label": mol, "color": color[i]})
        if amorph and ("dens_a" in strings[mol]):
                mc_pore = pa.diffusion.mc_profile(strings[mol]["mc_a"], len_step = [10,20,30,40,50,60], kwargs={"linestyle":"dotted", "color": color[i], "label": None})
        pore = pa.diffusion.mc_fit(strings[mol]["mc"], len_step = [10,20,30,40,50,60], section = [15,19.5], is_plot=False, is_print=False)
        res = pa.diffusion.mc_fit(strings[mol]["mc"], len_step = [10,20,30,40,50,60], section = [22.5,30.5], is_plot=False, is_print=False)
        ax = sns.lineplot(x=np.linspace(0,30.1,100),y=[mc_box[0] for i in range(100)], linestyle="--", label=None, color=color[i])
    plt.axvspan(xmin=23, xmax=31, facecolor="red", alpha=0.3, label = None)
    plt.axvspan(xmin=20, xmax=31, facecolor="grey", alpha=0.3, label = None)
    # Plot area


    if len(strings) != 1:
        plt.axvspan(xmin=0, xmax=10, facecolor="grey", alpha=0.3, label =  "Reservoir")
        plt.axvspan(xmin=15, xmax=19.7, facecolor="red", alpha=0.3, label = "Evaluated")
        plt.legend()
    elif len(strings) == 1:
        plt.axvspan(xmin=0, xmax=10, facecolor="grey", alpha=0.3, label = None)
        plt.axvspan(xmin=15, xmax=19.7, facecolor="red", alpha=0.3, label = None)
        plt.text(21.7, min(mc_pore[0]), 'NVT : D = ' + "%.2f" % (mc_box[1]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nRes  : D = " + "%.2f" % (res[0]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nPore : D = " + "%.2f" % (pore[0]) +"$\\cdot 10^{-9}\ \\frac{m^2}{s}$"  ,  bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 4})
        if amorph:
            legend = ["Pore System", "Box Diffusion", "Pore System (Amorph)",  "Reservoir", "Evaluated"]
        else:
            legend = ["Pore System", "Box Diffusion",  "Reservoir", "Evaluated"]
        plt.legend(legend,loc=1) 
        
    plt.tight_layout()
    plt.show()

In [19]:
# Pandas overwiev (groups)
def pandas_overview(strings, amorph):
    display(Markdown("# Results Overview"))
    #dens = []
    mol_vec = []
    dens_res = []
    mc_box = []
    pore = []
    res = [] 
    for mol in strings:
        # Density
        dens = pa.density.bins(strings[mol]["dens"], is_print=False)
        dens_res.append(dens["mean"]["ex"]*dens["sample"]["inp"]["mass"]*10/6.022)

        # Diffusion
        mc_box.append(round(pa.diffusion.mc_fit(strings[mol]["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)[0],2))
        pore.append(round( pa.diffusion.mc_fit(strings[mol]["mc"], len_step = [10,20,30,40,50,60], section = [15,19.5], is_plot=False, is_print=False)[0],2))
        res.append(round( pa.diffusion.mc_fit(strings[mol]["mc"], len_step = [10,20,30,40,50,60], section = [22.5,30.5], is_plot=False, is_print=False)[0],2))
        mol_vec.append(mol)
        if amorph and ("dens_a" in strings[mol]) and (len(strings) ==1):
            dens = pa.density.bins(strings[mol]["dens_a"], is_print=False)
            dens_res.append(dens["mean"]["ex"]*dens["sample"]["inp"]["mass"]*10/6.022)
            mc_box.append(round( pa.diffusion.mc_fit(strings[mol]["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)[0],2))
            pore.append(round( pa.diffusion.mc_fit(strings[mol]["mc_a"], len_step = [10,20,30,40,50,60], section = [15,19.5], is_plot=False, is_print=False)[0],2))
            res.append(round( pa.diffusion.mc_fit(strings[mol]["mc_a"], len_step = [10,20,30,40,50,60], section = [22.5,30.5], is_plot=False, is_print=False)[0],2))
            mol_vec.append(mol + " (Amorph)")
    data = { "Density": dens_res, "NVT": mc_box, "Reservoir": res, "Pore": pore, "Res/Pore": [round(res[i]/pore[i],2) for i in range(len(mol_vec))] , "Res/NVT": [round(res[i]/mc_box[i],2) for i in range(len(mol_vec))] }
    df = pd.DataFrame(data, index=[mol for mol in mol_vec])
    print(mc_box)
    display(Markdown("<div style=\"margin-left: auto;         margin-right: auto;            width: 30%\"> \n\n" + df.to_markdown() + "\n</div>"))

In [7]:
# Graphic user interface
def view_results(molecule,amorph):
    if (type(molecule) == list):
        strings = set_strings(molecule)
    else:
        strings = set_strings([molecule])
    if not (type(molecule) == list):
        display_molecule(strings)
    plot_density(strings, amorph)
    plot_mc_diffusion(strings, amorph)
    pandas_overview(strings, amorph) 

def update_drop(groups, check):
    dict = {}
    dict["alcohol"] = ["methanol","ethanol","1-propanol","1-butanol",]
    dict["aromatic"] = ["benzene","toluene","pyrrole","pyridine"]
    dict["alkene"] =   ["hexane","heptane","cyclopentane","cyclohexane",]
    dict["others"] =   ["water","tetrahydrofuran"]
    if check==False:
        drop = widgets.Dropdown(options=dict[groups]
        )   
        check_a = widgets.Checkbox(
        value=False,
        description='Display amorph results',
        )
        widgets.interact(view_results, molecule = drop, amorph = check_a)
    else:
        drop = dict[groups]
        check_a = False
        view_results(drop,check_a)



def dashboard():
    display(Markdown("# Overview   \n  The data set contains the following systems at a Temperature $T=295 K$: \n"))
    display(Markdown("<div style=\"margin-left: auto;         margin-right: auto;            width: 30%\"> \n\n |Alcohol   |Aromatic|Alkene      | Others |\n"    "|:-------------|:---------|:-----------|:-----------|\n"    "|Methanol  |Benzene |Cyclopentane|Water|\n"    "|Ethanol   |Toluene |Cyclohexane |Tetrahydrofuran|\n"    "|1-Propanol  |Pyrole  |Hexane      ||\n"    "|1-Butanol |Pyridine|Heptane    | |\n</div>"))
    
    display(Markdown("# Set molecule"))

    drop_group = widgets.RadioButtons(options=["alcohol","aromatic", "alkene", "others"]
    )

    check_group = widgets.Checkbox(
    value=False,
    description='Display group',
    )

    widgets.interact(update_drop, groups = drop_group, check=check_group)
    



In [20]:
# Display dashboard
dashboard()

# Overview   
  The data set contains the following systems at a Temperature $T=295 K$: 


<div style="margin-left: auto;         margin-right: auto;            width: 30%"> 

 |Alcohol   |Aromatic|Alkene      | Others |
|:-------------|:---------|:-----------|:-----------|
|Methanol  |Benzene |Cyclopentane|Water|
|Ethanol   |Toluene |Cyclohexane |Tetrahydrofuran|
|1-Propanol  |Pyrole  |Hexane      ||
|1-Butanol |Pyridine|Heptane    | |
</div>

# Set molecule

interactive(children=(RadioButtons(description='groups', options=('alcohol', 'aromatic', 'alkene', 'others'), …

interactive(children=(Dropdown(description='molecule', options=('methanol', 'ethanol', '1-propanol', '1-butano…